# Movie Analysis

In [2]:
import pandas as pd
import numpy as np
import networkx as nx

## Abstract
Cinema has become one of the highest profiting industries over the past century. The total box office revenue in North America alone amounted to $11.38 billion in 2016. With the possibility of great success, there is also a large risk of financial failure. This data exploration is motivated by answering the question what makes a movie successful. There is plenty of quantative data available for movies, such as the movies' budget, the release date, ratings etc., but in this analysis an attempt will be made to quantify movie information that is less measurable and then predict movie success.

## Introduction
Research has been done to determine what aspects of a movie make it more successful; however, much of this research is contradictory. The research paper "Early Predictions of Movie Success: the Who, What, and When of Profitability" states movies with a motion picture content rating 'R' will likely have lower profits, whereas the research paper "What Makes A Great Movie?" states a motion picture content rating 'R' will have higher a box-office. Both papers analyzed thousands of movies, but came to opposite conclusions. Some variables used to predict movie success in these studies, included budget, motion picture content rating, and actor popularity.

Based on these previous models, the dataset used will include movie title length, run-time, motion picture content rating, director, genre, release date, actors, an actor popularity score, average salary of the actors in the movie, budget, and opening weekend box-office revenue for predictor variables. The actor popularity score will be calculated from a network of actors connected through the movies they appeared in together and from the average actor income. Movie success will be determined by the awards it is nominated for, the awards it won, the MetaCritic score, the IMDb rating, and the profit of the movie.

## Data Scraped, Downloaded, Cleaned & Engineered
### Beginning Dataset
A beginning dataset is downloaded from IMDb with 10,000 movies, each entry containing the movie title, URL on IMDb rating, run-time, Year, Genres, Num Votes, Release Date, Directors. From this dataset, additional information on the movie budget, gross income, opening weekend box revenue, actors, Oscar nominations, Oscars won, other award nominations, other awards won, MetaCritic score, and content rating is scraped and cleaned. 
The data points will be collected from IMDb, which is a reputable source for information, according to their website, 

>"we [IMDb] actively gather information from and verify items with studios and 
filmmakers".

### Cleaning Data
After gathering each data point, the data set is complete, although the information is not clean or uniform. The first step to clean the data will be to remove all commas across each column in the DataFrame. Removing commas will make it easier to convert monetary amounts to ints. Next each date in the Release column will be changed to a pandas date object, which will simplify any calculations that rely on the release date of the movie. Each monetary amount will be converted into an int and converted into USD. Each unique genre will be made into a column, with a true or false boolean for each movie entry.
### Feature Engineering
To resolve the disagreement in monetary amounts, due to inflation, a dataset containing the CPI for each year from 1914 will used to adjust the monetary amounts. The CPI, Consumer Price Index, describes the amount of purchasing power the average consumer has. The length of the movie title will be added, and a NetworkX graph of all actors will be made. This network will connect nodes of actors to each other, if they appear in a movie together. The edges of the network will be weighted by the amount of movies the actors appear in together. An actor popularity score will be calculated from the actors appearing in the movie, based on how many other movies they appear in with other actors and the actor's income.

The total variables in the new dataset are movie title, title length, motion picture content rating, run-time, IMDb rating, genres, MetaCritic score, Oscar nominations, Oscar wins, other award nominations, other award wins, director, release date, budget, opening weekend, gross, profit, budget adjusted for inflation, opening weekend adjusted for inflation, gross adjusted for inflation, profit adjusted for inflation, the top ten actors in the movie, and actor popularity score. A separate network will hold the actor nodes and their connections.

In [3]:
df = pd.read_csv("Result_Data/total_engineered.csv",encoding = "ISO-8859-1")
del df["Unnamed: 0"]
df = df.fillna(df.mean())
print(list(df.columns))

['Actor_0', 'Actor_1', 'Actor_2', 'Actor_3', 'Actor_4', 'Actor_5', 'Actor_6', 'Actor_7', 'Actor_8', 'Actor_9', 'Budget', 'Directors', 'Gross', 'IMDb Rating', 'Meta Score', 'Num Votes', 'Opening Weekend', 'Oscar Nominations', 'Oscar Wins', 'Other Nominations', 'Other Wins', 'Release Date', 'Runtime (mins)', 'Title', 'Year', 'Genre: Short', 'Genre:  Comedy', 'Genre: Fantasy', 'Genre: Film-Noir', 'Genre: War', 'Genre: Musical', 'Genre:  Sport', 'Genre: Biography', 'Genre: Action', 'Genre:  Fantasy', 'Genre:  Animation', 'Genre:  Biography', 'Genre: Mystery', 'Genre:  Musical', 'Genre:  Romance', 'Genre: Thriller', 'Genre:  Film-Noir', 'Genre:  History', 'Genre: Western', 'Genre: Drama', 'Genre: Sci-Fi', 'Genre:  Horror', 'Genre: Romance', 'Genre: Adventure', 'Genre:  Family', 'Genre:  Sci-Fi', 'Genre: Animation', 'Genre:  Music', 'Genre: Music', 'Genre: History', 'Genre:  Mystery', 'Genre:  Thriller', 'Genre: Comedy', 'Genre:  Crime', 'Genre: Horror', 'Genre:  Drama', 'Genre:  War', 'Genr

In [4]:
df_x = df[['Actor_0', 'Actor_1', 'Actor_2', 'Actor_3', 'Actor_4', 'Actor_5', 'Actor_6', 'Actor_7', 'Actor_8', 'Actor_9', 
           'Budget', 'Directors', 'Release Date', 'Runtime (mins)', 'Title', 'Year', 'Genre: Short', 'Genre:  Comedy', 
           'Genre: Fantasy', 'Genre: Film-Noir', 'Genre: War', 'Genre: Musical', 'Genre:  Sport', 'Genre: Biography', 
           'Genre: Action', 'Genre:  Fantasy', 'Genre:  Animation', 'Genre:  Biography', 'Genre: Mystery', 
           'Genre:  Musical', 'Genre:  Romance', 'Genre: Thriller', 'Genre:  Film-Noir', 'Genre:  History', 
           'Genre: Western', 'Genre: Drama', 'Genre: Sci-Fi', 'Genre:  Horror', 'Genre: Romance', 'Genre: Adventure', 
           'Genre:  Family', 'Genre:  Sci-Fi', 'Genre: Animation', 'Genre:  Music', 'Genre: Music', 'Genre: History', 
           'Genre:  Mystery', 'Genre:  Thriller', 'Genre: Comedy', 'Genre:  Crime', 'Genre: Horror', 'Genre:  Drama', 
           'Genre:  War', 'Genre:  Western', 'Genre:  Adventure', 'Genre: Family', 'Genre:  Action', 'Genre: Crime', 
           'Content Rating: PASSED', 'Content Rating: TV-MA', 'Content Rating: X', 'Content Rating: NC-17', 
           'Content Rating: TV-14', 'Content Rating: M', 'Content Rating: GP', 'Content Rating: TV-PG', 
           'Content Rating: PG', 'Content Rating: PG-13', 'Content Rating: G', 'Content Rating: NR', 
           'Content Rating: APPROVED', 'Content Rating: UNRATED', 'Content Rating: M/PG', 'Content Rating: TV-13', 
           'Content Rating: NOT RATED', 'Content Rating: TV-G', 'Content Rating: R', 'Decade',  'Budget_Adjusted',  
           'Length of Title', 'Directors Prev Number Movies', 'Directors Prev Mean Profit', 'Directors Prev Mean IMDb',
           'Directors Prev Mean Meta', 'Directors Prev Mean Num Votes', 'Directors Prev Mean Nominations', 
           'Directors Prev Mean Wins', 'Actor Weights']]
df_y = df[['Gross', 'IMDb Rating', 'Meta Score', 'Num Votes', 'Oscar Nominations', 'Oscar Wins', 'Other Nominations', 
           'Other Wins','Profit','Gross_Adjusted',  'Profit_Adjusted', 'Profit_Bool', 'Total Nominations', 'Total Wins']]

In [5]:
actors_network = nx.read_yaml('Result_Data/network_of_actors.yaml')

In [6]:
# networkx.Graph.degree   weighted number of edges  https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.Graph.degree.html
weights = []
for edge in actors_network.edges():    
    weights.append(actors_network.get_edge_data(edge[0],edge[1])['weight'])
    
set(sorted(weights)[::-1])
#look into finding out who has 63 weights

{1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 19}

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

df_x_temp = df_x.select_dtypes(include=['float64','int','bool']).astype('float')
for n_components in [10,20,50,100]:
    for n_estimators in [10, 100, 500, 1000]:
        for criterion in ['mae', 'mse']:
            for max_features in ['auto', 'sqrt', 'log2']:
                RF = RFR(n_estimators = n_estimators, criterion = criterion, max_features = max_features)
                pca = PCA(n_components = n_components)
                df_x_temp2 = pca.fit_transform(df_x_temp)
                print("n_components: ", n_components,"\nn_estimators: ", n_estimators, "\ncriterion: ", criterion, "\nmax_features: ", max_features, "\n")
                print(cross_val_score(RF,df_x_temp2,df_y.select_dtypes(include=['float64','int','bool']).astype('float')),"\n")
                tr_x, tt_x, tr_y, tt_y = tts(df_x_temp2, df_y, test_size = .2)
                RF.fit(tr_x,tr_y)
                es_y = RF.predict(tt_x)
                np.array(tt_y.astype(float))[0:,0]
                print("\nMovie Gross Average Percent Error:")
                print((abs(es_y[0:,0]-np.array(tt_y.astype(float))[0:,0])/abs(np.array(tt_y.astype(float))[0:,0])).mean()*100,"%")
                print("\nIMDb Rating Average Percent Error:")
                print((abs(es_y[0:,1]-np.array(tt_y.astype(float))[0:,1])/abs(np.array(tt_y.astype(float))[0:,1])).mean()*100,"%")
                print("\nMeta Score Average Percent Error:")
                print((abs(es_y[0:,2]-np.array(tt_y.astype(float))[0:,2])/abs(np.array(tt_y.astype(float))[0:,2])).mean()*100,"%")
                print("\nNumber of Votes Average Percent Error:")
                print((abs(es_y[0:,3]-np.array(tt_y.astype(float))[0:,3])/abs(np.array(tt_y.astype(float))[0:,3])).mean()*100,"%")
                print("\nOscar Nominations Average Error:")
                print(abs(es_y[0:,4]-np.array(tt_y.astype(float))[0:,4]).mean())
                print("\nOscar Wins Average Error:")
                print(abs(es_y[0:,5]-np.array(tt_y.astype(float))[0:,5]).mean())
                print("\nOther Nominations Average Error:")
                print(abs(es_y[0:,6]-np.array(tt_y.astype(float))[0:,6]).mean())
                print("\nOther Wins Average Error:")
                print(abs(es_y[0:,7]-np.array(tt_y.astype(float))[0:,7]).mean())
                print("\nProfit Average Percent Error:")
                print((abs(es_y[0:,8]-np.array(tt_y.astype(float))[0:,8])/abs(np.array(tt_y.astype(float))[0:,8])).mean()*100,"%")
                print("\nGross Adjusted Average Percent Error:")
                print((abs(es_y[0:,9]-np.array(tt_y.astype(float))[0:,9])/abs(np.array(tt_y.astype(float))[0:,9])).mean()*100,"%")
                print("\nProfit Adjusted Average Percent Error:")
                print((abs(es_y[0:,10]-np.array(tt_y.astype(float))[0:,10])/abs(np.array(tt_y.astype(float))[0:,10])).mean()*100,"%")
                print("\nProfit Bool Average Error:")
                print(abs(es_y[0:,11]-np.array(tt_y.astype(float))[0:,11]).mean())
                print("\nTotal Nominations Average Error:")
                print(abs(es_y[0:,12]-np.array(tt_y.astype(float))[0:,12]).mean())
                print("\nTotal Wins Average Error:")
                print(abs(es_y[0:,13]-np.array(tt_y.astype(float))[0:,13]).mean())
                print("\n\n\n\n\n\n")



n_components:  10 
n_estimators:  10 
criterion:  mae 
max_features:  auto 

[-0.05913833 -0.08839107  0.05092325] 


Movie Gross Average Percent Error:
31628.6454466 %

IMDb Rating Average Percent Error:
12.0697001262 %

Meta Score Average Percent Error:
24.0247475764 %

Number of Votes Average Percent Error:
182.992145932 %

Oscar Nominations Average Error:
0.355155155155

Oscar Wins Average Error:
0.212012012012

Other Nominations Average Error:
8.93956456456

Other Wins Average Error:
6.03743743744

Profit Average Percent Error:
766.607576899 %

Gross Adjusted Average Percent Error:
36599.819403 %

Profit Adjusted Average Percent Error:
857.729286299 %

Profit Bool Average Error:
0.279304304304

Total Nominations Average Error:
9.09419419419

Total Wins Average Error:
6.19196696697







n_components:  10 
n_estimators:  10 
criterion:  mae 
max_features:  sqrt 

[-0.09894764 -0.05679197  0.0986815 ] 


Movie Gross Average Percent Error:
26924.0972533 %

IMDb Rating Average Percen

[ 0.03753364 -0.02106644  0.25041844] 


Movie Gross Average Percent Error:
30791.2861831 %

IMDb Rating Average Percent Error:
11.3397014257 %

Meta Score Average Percent Error:
21.7784638066 %

Number of Votes Average Percent Error:
177.981876293 %

Oscar Nominations Average Error:
0.361864364364

Oscar Wins Average Error:
0.202817817818

Other Nominations Average Error:
9.0384292626

Other Wins Average Error:
6.09905488822

Profit Average Percent Error:
716.471710301 %

Gross Adjusted Average Percent Error:
36400.1946085 %

Profit Adjusted Average Percent Error:
655.140281253 %

Profit Bool Average Error:
0.297802802803

Total Nominations Average Error:
9.21027010344

Total Wins Average Error:
6.23396229563







n_components:  10 
n_estimators:  500 
criterion:  mae 
max_features:  auto 

[-0.04195997 -0.06417311  0.15079509] 


Movie Gross Average Percent Error:
28028.7887869 %

IMDb Rating Average Percent Error:
11.6209218581 %

Meta Score Average Percent Error:
22.5647438061 %


In [ ]:
# importing all libraries to be used throughout data exploration
import re, json, requests, seaborn, warnings
warnings.filterwarnings( 'ignore' )
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from matplotlib import pyplot as plt, rcParams
%matplotlib inline
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from ipyparallel import Client
import time
def text_message(text,number):
    browser = webdriver.Chrome()
    url = "http://www.txtdrop.com/"
    browser.get( url )
    email = browser.find_element_by_id("emailfrom")
    email.send_keys("brookemosby@hotmail.com")
    first_3 = browser.find_element_by_id("npa")
    first_3.send_keys(number[:3])
    second_3 = browser.find_element_by_id("exchange")
    second_3.send_keys(number[3:6])
    last_4 = browser.find_element_by_id("number")
    last_4.send_keys(number[6:])
    message = browser.find_element_by_name("body")
    message.send_keys(text)
    browser.find_element_by_name("submit").click()
    browser.close()
text_message("Finished","6787995970")

In [ ]:
# Do some feature engineering with new actor dataset
# Add in column for popularity
# Need to make actor DataFrame?

In [ ]:
# Talk about machine learning methods, why there are good and suitable for our data

In [ ]:
# And now do some machine learning....

In [2]:
# Analyze ML models

In [ ]:
# Case study to see how well we predict on a specific movie

In [ ]:
# Insert some amazing conclusion here